In [35]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import *

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Batch_process_1")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# load data
# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

#  Google Storage File Path amazon
gsc_file_path = 'gs://data_de2024_2/amazon_prime_data.csv'  #  use your gcp bucket name. Also upload sales.csv first
# Create data frame
df1 = spark.read.format("csv").option("header", "true") \
       .load(gsc_file_path)
df1 = df1.withColumn('streaming_service', lit('Amazon'))
df1.printSchema()

#  Google Storage File Path disney
gsc_file_path = 'gs://data_de2024_2/disney_plus_data.csv'  #  use your gcp bucket name. Also upload sales.csv first
# Create data frame
df2 = spark.read.format("csv").option("header", "true") \
       .load(gsc_file_path)
df2 = df2.withColumn('streaming_service', lit('Disney'))

df2.printSchema()

#  Google Storage File Path netflix
gsc_file_path = 'gs://data_de2024_2/netflix_data.csv'  #  use your gcp bucket name. Also upload sales.csv first
# Create data frame
df3 = spark.read.format("csv").option("header", "true") \
       .load(gsc_file_path)
df3 = df3.withColumn('streaming_service', lit('Netflix'))
df3.printSchema()

# Concatenate all the dfs
df_all = df1.union(df2).union(df3)

# Show the schema of the concatenated DataFrame
df_all.printSchema()

# Select only the necessary columns
df_all = df_all.select('streaming_service', 'title', 'release_year')

# Change the datatypes of df_all
df_all = df_all.withColumn('release_year', col('release_year').cast('long'))

df_all.printSchema()

df_imdb = spark.read \
  .format("bigquery") \
  .load("de2024-assignment-2.IMDB_data.score_data")   
df_imdb = df_imdb.withColumnRenamed("title", "title1")
df_imdb.printSchema()
df_imdb.show(4)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)
 |-- streaming_service: string (nullable = false)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullab

In [36]:
# Join the 2 dataframes, inner
joinExpression = (df_imdb["title1"] == df_all['title']) & (df_imdb["releaseYear"] == df_all['release_year'])

df = df_all.join(df_imdb, joinExpression,"inner").drop('title1')
df.show(4)

+-----------------+------------------+------------+---------+-----+--------------------+-------------+--------+-----------+
|streaming_service|             title|release_year|       id| type|              genres|averageRating|numVotes|releaseYear|
+-----------------+------------------+------------+---------+-----+--------------------+-------------+--------+-----------+
|           Amazon|         Pollyanna|        1920|tt0011588|movie|Comedy, Drama, Fa...|          6.4|     527|       1920|
|           Amazon|One Exciting Night|        1922|tt0013458|movie|Comedy, Horror, M...|          5.3|     198|       1922|
|           Amazon|            Romola|        1924|tt0015289|movie|      Drama, History|          6.1|     257|       1924|
|           Amazon|       Tumbleweeds|        1925|tt0016461|movie|             Western|          6.5|     690|       1925|
+-----------------+------------------+------------+---------+-----+--------------------+-------------+--------+-----------+
only sho

## Best rated, and most rated movie, tvminiseries or tvseries per streaming service

In [37]:
from pyspark.sql import Row, Window

# 
windowasc1 = Window.partitionBy(col("streaming_service")).orderBy(col("averageRating").desc())
windowasc2 = Window.partitionBy(col("streaming_service")).orderBy(col("numVotes").desc())

df_windowed = df.withColumn("rank_rating", dense_rank().over(windowasc1)).withColumn("rank_votes", dense_rank().over(windowasc2))

result1 = df_windowed.where((col("rank_rating") == 1) | (col("rank_votes") == 1) ).select("*")

result = result1.withColumn("Popularity_Rating", when(col("rank_rating") == 1, "Highest rating")
                                                          .otherwise("Most voted")).select('streaming_service', 'type', 'title', 'release_year', 'Popularity_Rating', 'averageRating', 'numVotes')

result.show()

+-----------------+------------+--------------------+------------+-----------------+-------------+--------+
|streaming_service|        type|               title|release_year|Popularity_Rating|averageRating|numVotes|
+-----------------+------------+--------------------+------------+-----------------+-------------+--------+
|           Amazon|       movie| Catch Me If You Can|        2002|       Most voted|          8.1| 1131159|
|           Amazon|       movie|   L'Enfant Terrible|        2019|   Highest rating|          9.7|     256|
|           Disney|       movie|              Avatar|        2009|       Most voted|          7.9| 1409493|
|           Disney|tvMiniSeries|Cosmos: Possible ...|        2020|   Highest rating|          9.0|    8031|
|          Netflix|       movie|           Inception|        2010|       Most voted|          8.8| 2616043|
|          Netflix|tvMiniSeries|     Planet Earth II|        2016|   Highest rating|          9.5|  162312|
+-----------------+---------

In [38]:
# Use the Cloud Storage bucket for temporary BigQuery export data the connector uses.
bucket = "temp_de_2024_2"
spark.conf.set('temporaryGcsBucket', bucket)
# Saving the data to BigQuery
result.write.format('bigquery') \
  .option('table', 'de2024-assignment-2.Assignment_Data.Popular_Best_movies') \
  .mode("overwrite") \
  .save()

In [39]:
# Stop the spark context
spark.stop()